## TMDB 5000 Movie Dataset으로 장르기반 추천시스템
* 장르에 있는 텍스트를 벡터화 후 텍스트간 유사도를 구해 가까운 순서대로 정렬
* 유사도 척도 : 코사인유사도

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import koreanize_matplotlib

In [59]:
data = pd.read_csv('https://raw.githubusercontent.com/haram4th/ablearn/main/tmdb_5000_movies.csv')
data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [5]:
data.isna().sum() / len(data) * 100

budget                   0.000000
genres                   0.000000
homepage                64.355611
id                       0.000000
keywords                 0.000000
original_language        0.000000
original_title           0.000000
overview                 0.062461
popularity               0.000000
production_companies     0.000000
production_countries     0.000000
release_date             0.020820
revenue                  0.000000
runtime                  0.041641
spoken_languages         0.000000
status                   0.000000
tagline                 17.572351
title                    0.000000
vote_average             0.000000
vote_count               0.000000
dtype: float64

In [6]:
data = data.drop('homepage', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   id                    4803 non-null   int64  
 3   keywords              4803 non-null   object 
 4   original_language     4803 non-null   object 
 5   original_title        4803 non-null   object 
 6   overview              4800 non-null   object 
 7   popularity            4803 non-null   float64
 8   production_companies  4803 non-null   object 
 9   production_countries  4803 non-null   object 
 10  release_date          4802 non-null   object 
 11  revenue               4803 non-null   int64  
 12  runtime               4801 non-null   float64
 13  spoken_languages      4803 non-null   object 
 14  status                4803 non-null   object 
 15  tagline              

In [8]:
print(type(data['genres'][0]))
print(data['genres'][0])

<class 'str'>
[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]


In [13]:
data.columns

Index(['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [17]:
dict_cols = ['genres','keywords','production_companies','production_countries','spoken_languages']

In [9]:
import json

In [53]:
def dict_unpack(data):
    for col in dict_cols:
        temp = []
        for i in json.loads(data[col]):
            temp.append(i.get('name',None))
        temp = ' '.join(temp)
        data[col] = temp
    return data

In [60]:
data = data.apply(dict_unpack, axis=1)

In [49]:
data.columns

Index(['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [61]:
data2 = data[['original_title', 'genres', 'popularity', 'runtime', 'vote_average', 'vote_count']]

In [62]:
data2.head(1)

,original_title,genres,popularity,runtime,vote_average,vote_count
0,Avatar,Action Adventure Fantasy Science Fiction,150.437577,162.0,7.2,11800


In [63]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  4803 non-null   object 
 1   genres          4803 non-null   object 
 2   popularity      4803 non-null   float64
 3   runtime         4801 non-null   float64
 4   vote_average    4803 non-null   float64
 5   vote_count      4803 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 225.3+ KB


In [65]:
data2 = data2.dropna()

* 장르 컬럼의 텍스트를 숫자로 벡터화
* 코사인 유사도를 구해 비슷한 장르의 행 검색
* 유사도가 높은 행을 기준으로 내림차순 정렬

In [66]:
from sklearn.feature_extraction.text import CountVectorizer

In [71]:
cv = CountVectorizer(ngram_range=(1,2))
genres_mat = cv.fit_transform(data2['genres'])
# genres_mat.shape
# cv.get_feature_names_out()
genres_mat_df = pd.DataFrame(genres_mat)
genres_mat_df

,0
0,"(0, 0)\t1\n (0, 16)\t1\n (0, 124)\t1\n (0..."
1,"(0, 0)\t1\n (0, 16)\t1\n (0, 124)\t1\n (0..."
2,"(0, 0)\t1\n (0, 16)\t1\n (0, 1)\t1\n (0, ..."
3,"(0, 0)\t1\n (0, 64)\t1\n (0, 90)\t1\n (0,..."
4,"(0, 0)\t1\n (0, 16)\t1\n (0, 232)\t1\n (0..."
...,...
4796,"(0, 0)\t1\n (0, 64)\t1\n (0, 234)\t1\n (0..."
4797,"(0, 44)\t1\n (0, 214)\t1\n (0, 58)\t1"
4798,"(0, 90)\t1\n (0, 44)\t1\n (0, 214)\t1\n (..."
4799,


### 코사인 유사도를 이용해 genres_mat 유사도 산출 후 genres_similarity 생성
* 비슷한 장르의 영화를 찾기 위해 계산
* 코사인 유사도는 주로 문자, 문서의 유사도를 계산하는데 사용됨

In [69]:
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
genres_sim = cosine_similarity(genres_mat, genres_mat)
print(genres_sim.shape)

(4801, 4801)


In [73]:
genres_sim_df = pd.DataFrame(genres_sim)
genres_sim_df

,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,1.000000,0.596285,0.447214,0.125988,0.755929,0.596285,0.0,0.755929,0.447214,0.745356,...,0.000000,0.000000,0.000000,0.377964,0.000000,0.149071,0.0000,0.000000,0.0,0.0
1,0.596285,1.000000,0.400000,0.169031,0.338062,0.800000,0.0,0.338062,0.600000,0.800000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.0000,0.000000,0.0,0.0
2,0.447214,0.400000,1.000000,0.338062,0.507093,0.600000,0.0,0.507093,0.200000,0.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.0000,0.000000,0.0,0.0
3,0.125988,0.169031,0.338062,1.000000,0.142857,0.169031,0.0,0.142857,0.000000,0.169031,...,0.377964,0.169031,0.377964,0.428571,0.218218,0.676123,0.0000,0.125988,0.0,0.0
4,0.755929,0.338062,0.507093,0.142857,1.000000,0.507093,0.0,1.000000,0.169031,0.507093,...,0.000000,0.000000,0.000000,0.428571,0.000000,0.169031,0.0000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4796,0.149071,0.200000,0.400000,0.676123,0.169031,0.200000,0.0,0.169031,0.000000,0.200000,...,0.000000,0.200000,0.000000,0.169031,0.258199,1.000000,0.0000,0.000000,0.0,0.0
4797,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.258199,0.000000,0.000000,0.000000,0.000000,1.0000,0.384900,0.0,0.0
4798,0.000000,0.000000,0.000000,0.125988,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.333333,0.149071,0.333333,0.125988,0.000000,0.000000,0.3849,1.000000,0.0,0.0
4799,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.0


In [95]:
sorted_genres_sim = genres_sim.argsort()[:,::-1]

In [76]:
sorted_genres_sim[:1]

array([[   0,   14,  870, ..., 3871, 2339, 4800]])

In [79]:
data2.iloc[0,:]

original_title                                      Avatar
genres            Action Adventure Fantasy Science Fiction
popularity                                      150.437577
runtime                                              162.0
vote_average                                           7.2
vote_count                                           11800
Name: 0, dtype: object

In [80]:
data2.iloc[14,:]

original_title                                Man of Steel
genres            Action Adventure Fantasy Science Fiction
popularity                                       99.398009
runtime                                              143.0
vote_average                                           6.5
vote_count                                            6359
Name: 14, dtype: object

In [81]:
data2.iloc[870,:]

original_title                                 Superman II
genres            Action Adventure Fantasy Science Fiction
popularity                                       30.515175
runtime                                              127.0
vote_average                                           6.5
vote_count                                             629
Name: 870, dtype: object

## 영화 이름과 추천 받을 개수를 입력받아 출력하는 함수

In [82]:
data[data['original_title']=='Avatar']

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Ingenious Film Partners Twentieth Century Fox ...,United States of America United Kingdom,2009-12-10,2787965087,162.0,English Español,Released,Enter the World of Pandora.,Avatar,7.2,11800


In [83]:
sorted_genres_sim[0, 1:11] # 0번째는 자기자신

array([  14,  870,   46,  813, 3493, 1296, 1652,  419,  420, 1932])

In [96]:
data2.iloc[sorted_genres_sim[0, 1:11]]

,original_title,genres,popularity,runtime,vote_average,vote_count
14,Man of Steel,Action Adventure Fantasy Science Fiction,99.398009,143.0,6.5,6359
870,Superman II,Action Adventure Fantasy Science Fiction,30.515175,127.0,6.5,629
46,X-Men: Days of Future Past,Action Adventure Fantasy Science Fiction,118.078691,131.0,7.5,6032
813,Superman,Action Adventure Fantasy Science Fiction,48.507081,143.0,6.9,1022
3494,Beastmaster 2: Through the Portal of Time,Action Adventure Fantasy Science Fiction,1.478505,107.0,4.6,17
1296,Superman III,Comedy Action Adventure Fantasy Science Fiction,22.164202,125.0,5.3,490
1652,Dragonball Evolution,Action Adventure Fantasy Science Fiction Thriller,21.677732,85.0,2.9,462
419,Jumper,Adventure Fantasy Science Fiction,21.218000,88.0,5.9,1799
420,Hellboy II: The Golden Army,Adventure Fantasy Science Fiction,58.579760,120.0,6.5,1527
1932,Sheena,Action Adventure Comedy Fantasy Science Fiction,4.020194,117.0,5.0,22


In [100]:
def find_sim_movie(data, sorted_genres_sim, title_name, top_n=10):
    title_movie = data[data['original_title'] == title_name]
    title_index = title_movie.index.values[0]
    similar_idxs = sorted_genres_sim[title_index, 1:top_n+1]
    return data.iloc[similar_idxs]

In [101]:
find_sim_movie(data2, sorted_genres_sim, 'Avatar', 20)

,original_title,genres,popularity,runtime,vote_average,vote_count
14,Man of Steel,Action Adventure Fantasy Science Fiction,99.398009,143.0,6.5,6359
870,Superman II,Action Adventure Fantasy Science Fiction,30.515175,127.0,6.5,629
46,X-Men: Days of Future Past,Action Adventure Fantasy Science Fiction,118.078691,131.0,7.5,6032
813,Superman,Action Adventure Fantasy Science Fiction,48.507081,143.0,6.9,1022
3494,Beastmaster 2: Through the Portal of Time,Action Adventure Fantasy Science Fiction,1.478505,107.0,4.6,17
1296,Superman III,Comedy Action Adventure Fantasy Science Fiction,22.164202,125.0,5.3,490
1652,Dragonball Evolution,Action Adventure Fantasy Science Fiction Thriller,21.677732,85.0,2.9,462
419,Jumper,Adventure Fantasy Science Fiction,21.218000,88.0,5.9,1799
420,Hellboy II: The Golden Army,Adventure Fantasy Science Fiction,58.579760,120.0,6.5,1527
1932,Sheena,Action Adventure Comedy Fantasy Science Fiction,4.020194,117.0,5.0,22
